In [1]:
from tools.data_reading import CompressedWindowDataset, CocoWindowDataset
import torch
from torch import nn
from torchvision import transforms
from torch.utils.data import DataLoader
from tools import utils
from tools.engine import train_one_epoch, evaluate
from model_zoo.InputInjection import InputInjection
from torch.utils.tensorboard import SummaryWriter

In [2]:
dataset = CocoWindowDataset()

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


In [3]:
noneC = 0
for i in range(len(dataset)):
    img, spectr, lab = dataset[i]
    if img is None:
        noneC += 1
print(noneC, len(dataset))

[{'id': 1, 'image_id': 1, 'category_id': 1, 'bbox': [1.0, 0.0, 126.0, 95.0], 'iscrowd': 0, 'area': 11970.0}]
[{'id': 2, 'image_id': 2, 'category_id': 1, 'bbox': [1.0, 1.0, 126.0, 94.0], 'iscrowd': 0, 'area': 11844.0}]
[{'id': 3, 'image_id': 3, 'category_id': 1, 'bbox': [1.0, 0.0, 127.0, 94.0], 'iscrowd': 0, 'area': 11938.0}]
[{'id': 4, 'image_id': 4, 'category_id': 1, 'bbox': [0.0, 1.0, 127.0, 94.0], 'iscrowd': 0, 'area': 11938.0}]
[{'id': 5, 'image_id': 5, 'category_id': 1, 'bbox': [1.0, 0.0, 126.0, 94.0], 'iscrowd': 0, 'area': 11844.0}]
[{'id': 6, 'image_id': 6, 'category_id': 1, 'bbox': [1.0, 0.0, 127.0, 94.0], 'iscrowd': 0, 'area': 11938.0}]
[{'id': 7, 'image_id': 7, 'category_id': 1, 'bbox': [1.0, 0.0, 126.0, 94.0], 'iscrowd': 0, 'area': 11844.0}]
[{'id': 8, 'image_id': 8, 'category_id': 1, 'bbox': [1.0, 0.0, 127.0, 95.0], 'iscrowd': 0, 'area': 12065.0}]
[{'id': 9, 'image_id': 9, 'category_id': 1, 'bbox': [1.0, 1.0, 127.0, 95.0], 'iscrowd': 0, 'area': 12065.0}]
[{'id': 10, 'image_

In [4]:
train = DataLoader(dataset,batch_size = 1, shuffle=True, collate_fn=utils.collate_fn)

In [5]:
model = InputInjection()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'running on {device}')
if device == 'cuda':
    device_id = torch.cuda.current_device()
    print(f'using gpu {torch.cuda.get_device_name(device_id)}')
model.to(device)
writer = SummaryWriter()
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params,lr=0.00003)

running on cpu


In [7]:
evaluate(writer, model, train, device, print_freq=100)

[{'id': 336, 'image_id': 590, 'category_id': 1, 'bbox': [1.0, 0.0, 113.0, 95.0], 'iscrowd': 0, 'area': 10735.0}]


In [6]:
num_epochs = 10
for epoch in range(num_epochs):
    train_one_epoch(writer, model, optimizer, train, device, epoch, print_freq=100)
    evaluate(writer, model, train, device, print_freq=100)
    torch.save(model.state_dict(), f'inputinjection-train-1.weights')

[{'id': 903, 'image_id': 1485, 'category_id': 1, 'bbox': [5.0, 0.0, 115.0, 95.0], 'iscrowd': 0, 'area': 10925.0}]


/Users/michaelyhuang/ProjectData/python/virtual-environments/ml-venv/lib/python3.9/site-packages/torch/nn/functional.py:3609: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
/Users/michaelyhuang/ProjectData/python/virtual-environments/ml-venv/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: [0]  [   0/2197]  eta: 8:02:40  lr: 0.000030  loss: 1.5170 (1.5170)  loss_classifier: 0.6955 (0.6955)  loss_box_reg: 0.0000 (0.0000)  loss_objectness: 0.6989 (0.6989)  loss_rpn_box_reg: 0.1226 (0.1226)  time: 13.1820  data: 0.0063
[{'id': 184, 'image_id': 410, 'category_id': 1, 'bbox': [37.0, 4.0, 91.0, 91.0], 'iscrowd': 0, 'area': 8281.0}]
[{'id': 226, 'image_id': 494, 'category_id': 1, 'bbox': [32.0, 25.0, 68.0, 71.0], 'iscrowd': 0, 'area': 4828.0}]
[{'id': 80, 'image_id': 65, 'category_id': 1, 'bbox': [35.0, 11.0, 56.0, 85.0], 'iscrowd': 0, 'area': 4760.0}, {'id': 81, 'image_id': 65, 'category_id': 1, 'bbox': [109.0, 6.0, 18.0, 33.0], 'iscrowd': 0, 'area': 594.0}]
[{'id': 460, 'image_id': 1000, 'category_id': 1, 'bbox': [1.0, 9.0, 53.0, 86.0], 'iscrowd': 0, 'area': 4558.0}, {'id': 461, 'image_id': 1000, 'category_id': 1, 'bbox': [66.0, 34.0, 19.0, 16.0], 'iscrowd': 0, 'area': 304.0}, {'id': 462, 'image_id': 1000, 'category_id': 1, 'bbox': [66.0, 66.0, 29.0, 29.0], 'iscrowd': 

KeyboardInterrupt: 